In [0]:
dbutils.fs.ls("/")

[FileInfo(path='dbfs:/Volumes/', name='Volumes/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/Workspace/', name='Workspace/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/databricks-datasets/', name='databricks-datasets/', size=0, modificationTime=0)]

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS learning;
CREATE SCHEMA IF NOT EXISTS learning.delta;


In [0]:
%sql
CREATE VOLUME IF NOT EXISTS learning.delta.my_volume;


In [0]:
dbutils.fs.ls("/Volumes/learning/delta/my_volume")

[]

In [0]:
dbutils.fs.mkdirs("/Volumes/learning/delta/my_volume/sales")

True

In [0]:
data = [(1, "A", 100), (2, "B", 200)]
df = spark.createDataFrame(data, ["id", "name", "price"])

In [0]:
df.write.format("delta") \
  .save("/Volumes/learning/delta/my_volume/sales")

In [0]:
spark.read.format("delta") \
  .load("/Volumes/learning/delta/my_volume/sales") \
  .show()

+---+----+-----+
| id|name|price|
+---+----+-----+
|  1|   A|  100|
|  2|   B|  200|
+---+----+-----+



In [0]:
dbutils.fs.ls("/Volumes/learning/delta/my_volume/sales")

[FileInfo(path='dbfs:/Volumes/learning/delta/my_volume/sales/_delta_log/', name='_delta_log/', size=0, modificationTime=1765695572677),
 FileInfo(path='dbfs:/Volumes/learning/delta/my_volume/sales/part-00000-2423059d-a3f7-4304-b683-9ff572f4a374.c000.snappy.parquet', name='part-00000-2423059d-a3f7-4304-b683-9ff572f4a374.c000.snappy.parquet', size=1025, modificationTime=1765695011000)]

In [0]:
dbutils.fs.ls("/Volumes/learning/delta/my_volume/sales/_delta_log")

[FileInfo(path='dbfs:/Volumes/learning/delta/my_volume/sales/_delta_log/00000000000000000000.crc', name='00000000000000000000.crc', size=2844, modificationTime=1765695012000),
 FileInfo(path='dbfs:/Volumes/learning/delta/my_volume/sales/_delta_log/00000000000000000000.json', name='00000000000000000000.json', size=1781, modificationTime=1765695012000),
 FileInfo(path='dbfs:/Volumes/learning/delta/my_volume/sales/_delta_log/_staged_commits/', name='_staged_commits/', size=0, modificationTime=1765695739981)]

In [0]:
spark.read.format("delta") \
  .load("/Volumes/learning/delta/my_volume/sales") \
  .show()


+---+----+-----+
| id|name|price|
+---+----+-----+
|  1|   A|  100|
|  2|   B|  200|
+---+----+-----+



In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(
    spark,
    "/Volumes/learning/delta/my_volume/sales"
)

delta_table.update(
    condition="id = 1",
    set={"price": "150"}
)


DataFrame[num_affected_rows: bigint]

In [0]:
spark.read.format("delta") \
  .load("/Volumes/learning/delta/my_volume/sales") \
  .show()


+---+----+-----+
| id|name|price|
+---+----+-----+
|  1|   A|  150|
|  2|   B|  200|
+---+----+-----+



In [0]:
spark.read.format("delta") \
  .option("versionAsOf", 0) \
  .load("/Volumes/learning/delta/my_volume/sales") \
  .show()


+---+----+-----+
| id|name|price|
+---+----+-----+
|  1|   A|  100|
|  2|   B|  200|
+---+----+-----+



In [0]:
spark.read.format("delta") \
  .load("/Volumes/learning/delta/my_volume/sales") \
  .show()

+---+----+-----+
| id|name|price|
+---+----+-----+
|  1|   A|  150|
|  2|   B|  200|
+---+----+-----+



In [0]:
delta_table.history().show()

+-------+-------------------+--------------+--------------------+---------+--------------------+----+-----------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|        userId|            userName|operation| operationParameters| job|         notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+--------------+--------------------+---------+--------------------+----+-----------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      2|2025-12-14 07:12:09|74454656808731|abdullahjubairt@g...| OPTIMIZE|{clusterBy -> [],...|NULL|{564744395035721}|1214-063241-67slj...|          1|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Databricks-Runtim...|
|      1|2025-12-14 07:12:06|74454656808

In [0]:
from delta.tables import DeltaTable

path = "/Volumes/learning/delta/my_volume/sales"

delta_table = DeltaTable.forPath(spark, path)


In [0]:
delta_table.update(
    condition="id = 1",
    set={"price": "300"}
)


DataFrame[num_affected_rows: bigint]

In [0]:
delta_table.delete(
    condition="id = 2"
)


DataFrame[num_affected_rows: bigint]

In [0]:
spark.read.format("delta").load(path).show()

+---+----+-----+
| id|name|price|
+---+----+-----+
|  1|   A|  300|
+---+----+-----+



In [0]:
spark.read.format("delta") \
  .option("versionAsOf", 0) \
  .load("/Volumes/learning/delta/my_volume/sales") \
  .show()

+---+----+-----+
| id|name|price|
+---+----+-----+
|  1|   A|  100|
|  2|   B|  200|
+---+----+-----+



In [0]:
spark.read.format("delta") \
  .option("versionAsOf", 1) \
  .load("/Volumes/learning/delta/my_volume/sales") \
  .show()


+---+----+-----+
| id|name|price|
+---+----+-----+
|  2|   B|  200|
|  1|   A|  150|
+---+----+-----+



In [0]:
spark.read.format("delta") \
  .option("versionAsOf", 2) \
  .load("/Volumes/learning/delta/my_volume/sales") \
  .show()


+---+----+-----+
| id|name|price|
+---+----+-----+
|  1|   A|  150|
|  2|   B|  200|
+---+----+-----+



In [0]:
spark.read.format("delta") \
  .option("versionAsOf", 3) \
  .load("/Volumes/learning/delta/my_volume/sales") \
  .show()


+---+----+-----+
| id|name|price|
+---+----+-----+
|  2|   B|  200|
|  1|   A|  300|
+---+----+-----+



In [0]:
spark.read.format("delta") \
  .option("versionAsOf", 4) \
  .load("/Volumes/learning/delta/my_volume/sales") \
  .show()


+---+----+-----+
| id|name|price|
+---+----+-----+
|  1|   A|  300|
|  2|   B|  200|
+---+----+-----+



In [0]:
spark.read.format("delta") \
  .option("versionAsOf", 5) \
  .load("/Volumes/learning/delta/my_volume/sales") \
  .show()


+---+----+-----+
| id|name|price|
+---+----+-----+
|  1|   A|  300|
+---+----+-----+



In [0]:
spark.read.format("delta") \
  .option("versionAsOf", 6) \
  .load("/Volumes/learning/delta/my_volume/sales") \
  .show()


+---+----+-----+
| id|name|price|
+---+----+-----+
|  1|   A|  300|
+---+----+-----+



In [0]:
spark.read.format("delta") \
  .option("versionAsOf", 7) \
  .load("/Volumes/learning/delta/my_volume/sales") \
  .show()


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7506870374843630>, line 4
      1 spark.read.format("delta") \
      2   .option("versionAsOf", 7) \
      3   .load("/Volumes/learning/delta/my_volume/sales") \
----> 4   .show()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1123, in DataFrame.show(self, n, truncate, vertical)
   1122 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
-> 1123     print(self._show_string(n, truncate, vertical))

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:876, in DataFrame._show_string(self, n, truncate, vertical)
    859     except ValueError:
    860         raise PySparkTypeError(
    861             errorClass="NOT_BOOL",
    862             messageParameters={
   (...)
    865             },
    866         )